<a href="https://colab.research.google.com/github/riinakik/digital-humanities-technologies/blob/main/assignment_01_stanza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introduction**

For this assignment, I selected a short biographical text about the French composer Claude Debussy. The goal of the analysis was to explore the linguistic structure of the text using the Stanza natural language processing toolkit. Specifically, I examined how the text is organized at different linguistic levels: sentence structure, tokenization, parts of speech, lemmas, dependency relations, and named entities.
I wanted to understand the text’s writing style, its distribution of parts of speech, and the proportions of nouns, verbs, and other grammatical categories. In addition, I analyzed the morphological and syntactic patterns to see how biographical information is presented through language.

In [ ]:
# Change directory to the assignment folder in Google Drive.
%cd "/content/drive/MyDrive/Digihumanitaaria tehnoloogiad/assignment_debussy"

# Open the text file containing the Claude Debussy biography.
# "r" means reading mode and UTF-8 ensures correct handling of special characters.
content = open("/content/drive/MyDrive/Digihumanitaaria tehnoloogiad/assignment_debussy/debussy.txt",
               "r", encoding="utf-8").read()

# Display the loaded text to verify that the file was successfully read.
content


**Result**

Opens and reads the content of the debussy.txt file

In [ ]:
# Install the Stanza NLP library.
!pip install stanza

# Import the Stanza module and download the English language models.
import stanza
stanza.download("en")

# Initialize the Stanza NLP pipeline for English.
nlp = stanza.Pipeline("en")

# Load the selected text file ("debussy.txt") into a Python string.
# UTF-8 encoding ensures that special characters are handled correctly.
content = open("debussy.txt", "r", encoding="utf-8").read()

# Process the text using the Stanza pipeline.
# This creates a 'doc' object that contains sentences, words, POS tags, lemmas, and more.
doc = nlp(content)



In [ ]:
# Print the number of sentences in the processed document.
len(doc.sentences)


**Result**

There are 14 sentences in this text

In [ ]:
# Display the full text of the first sentence.
doc.sentences[0].text

**Result**

Shows the first sentence of the text: "Achille Claude Debussy (22 August 1862 – 25 March 1918) was a French composer."

In [ ]:
# Extract and print all individual word tokens from the first sentence.
[w.text for w in doc.sentences[0].words]


**Result**

The output shows all individual tokens (words and symbols) from the first sentence of the text.

In [ ]:
# Extract all word tokens from the entire text, excluding punctuation.
# This list collects every meaningful word Stanza identifies in all sentences.
# We remove punctuation because it does not contribute to linguistic analysis.
words = [w.text for s in doc.sentences for w in s.words if w.upos != "PUNCT"]

# Display the total number of non-punctuation words in the text.
len(words)


**Result**

The output shows the total number (276) of meaningful words in the text after removing punctuation.


In [ ]:
# Extract all nouns from the entire text.
# Nouns are identified by POS tags starting with "NN" (e.g., NN, NNS, NNP, NNPS).
nouns = [w.text for s in doc.sentences for w in s.words if w.xpos.startswith("NN")]

# Extract all verbs from the entire text.
# Verbs are identified by POS tags starting with "VB" (e.g., VB, VBD, VBG, VBN, VBP, VBZ).
verbs = [w.text for s in doc.sentences for w in s.words if w.xpos.startswith("VB")]

# Display the total number of nouns and verbs.
len(nouns), len(verbs)

**Result**

A high noun count (80) vs verbs (33) shows that the text is descriptive and informational, which is typical of a biography.
A lower verb count confirms that the text focuses on facts, descriptions, dates, and names, rather than actions or events.

In [ ]:
# Calculate the proportion of verbs relative to all meaningful words in the text.
# This shows how verb-heavy the text is (amount of action or events described).
len(verbs) / len(words)


**Result**

This means that approximately 11.96% of all meaningful words in the text are verbs.

In [ ]:
# Define a dictionary that groups POS categories and their corresponding tag codes.
# This allows us to count how many times each part of speech appears in the text.
pos_tags = {
    "Conjunction": ["CC"],
    "Pronoun": ["PRP", "PRP$", "WP", "WP$"],
    "Noun": ["NN", "NNS", "NNP", "NNPS"],
    "Verb": ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"],
    "Adjective": ["JJ", "JJR", "JJS"]
}

# Count how many words belong to each POS category defined above.
# The result is stored in a dictionary where each key is a POS name
# and each value is the total count of that category in the text.
results = {}
for pos_name, tags in pos_tags.items():
    results[pos_name] = len([w.text for s in doc.sentences for w in s.words if w.xpos in tags])

# Display the POS frequency counts.
results


**Result**

The POS distribution shows that the text is descriptive, fact-based, and biography-style, with a strong focus on nouns and adjectives rather than actions.

In [ ]:
# Sort the POS results by their frequency in descending order.
# This makes it easy to see which parts of speech appear most often in the text.
sorted(results.items(), key=lambda x: x[1], reverse=True)


**Result**

The distribution is noun-heavy, confirming that the text focuses on presenting information rather than narrating events or dialogue.

In [ ]:
# Extract the lemma (dictionary form) of every word in the text.
lemmas = [w.lemma for s in doc.sentences for w in s.words]

# Display the first 40 lemmas.
lemmas[:40]


**Result**

The output shows the first 40 lemmas of the text.

A lemma is the base or dictionary form of a word. For example:
"was" → "be"
"composers" → "composer"

In [ ]:
# Generate a list of tuples showing the dependency relations in the first sentence.
# For each word, we extract:
# 1) the word form (w.text)
# 2) the dependency label (w.deprel), showing the grammatical function
# 3) the head word it depends on (syntactic governor)
#
# If w.head == 0, the word is the ROOT of the sentence.
# Otherwise, w.head-1 gives the index of its governing word.
[(w.text, w.deprel, doc.sentences[0].words[w.head-1].text if w.head > 0 else "ROOT")
 for w in doc.sentences[0].words]


**Result**

The output shows the dependency relations for every word in the first sentence.

For example:

('Achille', 'nsubj', 'composer')
→ Achille is the subject of the verb phrase headed by composer.

('Claude', 'flat', 'Achille')
→ Claude is linked to Achille as part of a name construction.

In [ ]:
# Create a Stanza pipeline with specific processors enabled:
# - tokenize: split text into words
# - pos: assign part-of-speech tags
# - lemma: reduce words to their base form
# - depparse: analyze syntactic dependency structure
# - ner: identify named entities (people, locations, dates, organizations, etc.)
nlp = stanza.Pipeline("en", processors="tokenize,pos,lemma,depparse,ner")

# Process the text using the full pipeline.
doc = nlp(content)

# Extract all named entities from the text.
# For each entity, we store:
# 1) the entity text (ent.text)
# 2) the entity type label (ent.type), such as PERSON, DATE, ORG, GPE (location), WORK_OF_ART, etc.
[(ent.text, ent.type) for ent in doc.ents]


**Result**

In this text, Stanza successfully identifies important biographical information about Claude Debussy.

For example:
“Achille Claude Debussy” → PERSON.
“22 August 1862” → DATE.
“the Conservatoire de Paris” → ORG.

In [ ]:
# For every word in the first sentence, extract three pieces of information:
# 1) w.text  → the original word form in the sentence
# 2) w.xpos  → the detailed POS tag (Penn Treebank / universal POS extension)
# 3) w.feats → morphological features, such as Number, Tense, Person, Mood, Case, Gender, etc.

# This gives a detailed grammatical profile of each word,
# allowing deeper analysis of how the sentence is structured linguistically.
[(w.text, w.xpos, w.feats) for w in doc.sentences[0].words]


**Result**

This output provides a deep grammatical breakdown of each token in the sentence.
It shows:

Names (e.g., Achille, Claude, Debussy) marked as NNP, Number=Sing.

Dates and numbers correctly recognized as numerals (CD, NumForm=Digit).

Verbs with rich grammatical detail (e.g., was → VBD, with features indicating past tense).

Punctuation and symbols categorized appropriately.

Nouns and adjectives tagged with their syntactic and morphological information.

In [ ]:
# ------------------------------
# ANALYSIS PART
# ------------------------------

num_sentences = len(doc.sentences)
words = [w.text for s in doc.sentences for w in s.words if w.upos != "PUNCT"]
nouns = [w.text for s in doc.sentences for w in s.words if w.xpos.startswith("NN")]
verbs = [w.text for s in doc.sentences for w in s.words if w.xpos.startswith("VB")]
verb_ratio = len(verbs) / len(words)

pos_tags = {
    "Conjunction": ["CC"],
    "Pronoun": ["PRP", "PRP$", "WP", "WP$"],
    "Noun": ["NN", "NNS", "NNP", "NNPS"],
    "Verb": ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"],
    "Adjective": ["JJ", "JJR", "JJS"]
}

results = {}
for pos_name, tags in pos_tags.items():
    results[pos_name] = len([w for s in doc.sentences for w in s.words if w.xpos in tags])

# ------------------------------
# 4. CREATE OUTPUT FOLDER
# ------------------------------

output_folder = "/content/drive/MyDrive/Digihumanitaaria tehnoloogiad/assignment_debussy/output"
os.makedirs(output_folder, exist_ok=True)

output_file = os.path.join(output_folder, "analysis.txt")

# ------------------------------
# 5. WRITE ANALYSIS TO FILE
# ------------------------------

with open(output_file, "w", encoding="utf-8") as out:
    out.write("Analysis of the Selected Text\n")
    out.write("------------------------------------\n\n")

    out.write(f"Number of sentences: {num_sentences}\n")
    out.write(f"Total meaningful words (no punctuation): {len(words)}\n")
    out.write(f"Noun count: {len(nouns)}\n")
    out.write(f"Verb count: {len(verbs)}\n")
    out.write(f"Proportion of verbs: {verb_ratio:.2f}\n\n")

    out.write("Part-of-Speech distribution:\n")
    for pos_name, count in results.items():
        out.write(f"  {pos_name}: {count}\n")

    out.write("\nNamed Entities:\n")
    for ent in doc.ents:
        out.write(f"  {ent.text}  -->  {ent.type}\n")

print("analysis.txt has been created successfully!")
print("Saved to:", output_file)


**Result**

This code generates a new folder and creates an output file named analysis.txt inside it.

The file contains a summary of the linguistic features extracted from the Claude Debussy biography using the Stanza NLP pipeline.